In [1]:
import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape
import pprint
import json
import xmltodict
import pandas as pd
 

# file='resources/horsies.xml'
file='resources/DMR20180722TCH.xml'
tree = ET.parse(file)
root = tree.getroot()
print(root)

<Element 'CHART' at 0x00000188261A5868>


In [2]:
with open(file, 'r') as f:
    xmlString = f.read()
 
# print("XML input (sample.xml):")
# print(xmlString)
     
jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
 
print("\nJSON output(output.json):")
print(jsonString)


JSON output(output.json):
{
    "CHART": {
        "@RACE_DATE": "2018-07-22",
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "@xsi:noNamespaceSchemaLocation": "https://info.trackmaster.com/xmlSchema/tchSchema.xsd",
        "TRACK": {
            "CODE": "DMR",
            "NAME": "DEL MAR"
        },
        "RACE": [
            {
                "@NUMBER": "1",
                "BREED": "TB",
                "CARD_ID": "D",
                "TYPE": "Maiden Special Weight",
                "PURSE": "60000",
                "RACE_TEXT": "DEL MAR RACE    1  6F Dirt CR: 99. Maiden Special Weight Purse 60000. FOR MAIDENS, THREE YEARS OLD  AND UPWARD. Three Year Olds, 120 lbs.; Older, 124 lbs.",
                "AGE_RESTRICTIONS": "3U",
                "DISTANCE": "600",
                "DIST_UNIT": "F",
                "ABOUT_DIST_FLAG": null,
                "COURSE_ID": "D",
                "COURSE_DESC": "Dirt",
                "SURFACE": "D",
              

In [3]:
data=json.loads(jsonString)
data=data['CHART']['RACE']
horsieData=[]
for race in data:
    run={}
    run['distance']=float(race['DISTANCE'])
    run['fr1']=float(race['FRACTION_1'])
    run['fr2']=float(race['FRACTION_2'])
    run['fr3']=float(race['FRACTION_3'])
    run['fr4']=float(race['FRACTION_4'])
    run['fr5']=float(race['FRACTION_5'])
    run['frw']=float(race['WIN_TIME'])
    horse={}
    for entry in race['ENTRY']:
        horse['pp_start']=int(entry['START_POSITION'])
        horse['pp_finish']=int(entry['OFFICIAL_FIN'])
        horse['age']=int(entry['AGE'])
        horse['speed_rating']=int(entry['SPEED_RATING'])
        horse['jockey']=str(entry['JOCKEY']['KEY'])
        horse['trainer']=str(entry['TRAINER']['KEY'])
        horse['odds']=float(entry['DOLLAR_ODDS'])
        call={}
        for poc in entry['POINT_OF_CALL']:
            if(poc['@WHICH']=='1'):
                call['poc1']=float(poc['LENGTHS'])
#                 row['poc1p']=int(poc['POSITION'])
            elif(poc['@WHICH']=='2'):
                call['poc2']=float(poc['LENGTHS'])
#                 row['poc2p']=int(poc['POSITION'])
            elif(poc['@WHICH']=='3'):
                call['poc3']=float(poc['LENGTHS'])
#                 row['poc3p']=int(poc['POSITION'])
            elif(poc['@WHICH']=='4'):
                call['poc4']=float(poc['LENGTHS'])
#                 row['poc4p']=int(poc['POSITION'])
            elif(poc['@WHICH']=='5'):
                call['poc5']=float(poc['LENGTHS'])
#                 row['poc5p']=int(poc['POSITION'])
            elif(poc['@WHICH']=='FINAL'):
                call['pocf']=float(poc['LENGTHS'])
#                 row['pocfp']=int(poc['POSITION'])
        outp={**run, **horse, **call}
        horsieData.append(outp)
# pprint.pprint(horsieData)
df=pd.DataFrame(horsieData)
df.head()

,age,distance,fr1,fr2,fr3,fr4,fr5,frw,jockey,odds,poc1,poc2,poc3,poc4,poc5,pocf,pp_finish,pp_start,speed_rating,trainer
0,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,161943,96.3,4.7,5.1,0.0,0.0,8.6,15.0,7,6,69,267241
1,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,74725,35.8,10.7,8.6,0.0,0.0,7.6,7.5,4,7,86,518
2,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,122644,0.5,0.1,0.0,0.0,0.0,0.0,0.5,2,4,101,83
3,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,114312,2.9,1.7,1.6,0.0,0.0,0.1,0.0,1,1,102,26392
4,4,600.0,22.32,46.07,58.61,0.0,0.0,70.78,135108,13.1,0.6,1.1,0.0,0.0,2.6,8.8,6,5,83,3794


In [4]:
horseLength=8
furlong=660
dist1=1320
dist2=2640
dist3=3960
df['speed1']=((dist1-(horseLength*df['poc1'].astype(float)))/df['fr1'].astype(float)).round(2)
df['speed2']=((dist2-(horseLength*df['poc2'].astype(float)))/df['fr2'].astype(float)).round(2)
# df['speed3']=((dist3-(horseLength*df['poc3'].astype(float)))/df['fr3'].astype(float)).round(2)
df['speedf']=(((df['distance'].astype(float)/100*furlong)-(horseLength*df['pocf'].astype(float)))/df['frw'].astype(float)).round(2)
df.head()

,age,distance,fr1,fr2,fr3,fr4,fr5,frw,jockey,odds,...,poc4,poc5,pocf,pp_finish,pp_start,speed_rating,trainer,speed1,speed2,speedf
0,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,161943,96.3,...,0.0,8.6,15.0,7,6,69,267241,57.46,56.42,54.25
1,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,74725,35.8,...,0.0,7.6,7.5,4,7,86,518,55.30,55.81,55.10
2,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,122644,0.5,...,0.0,0.0,0.5,2,4,101,83,59.10,57.30,55.89
3,3,600.0,22.32,46.07,58.61,0.0,0.0,70.78,114312,2.9,...,0.0,0.1,0.0,1,1,102,26392,58.53,57.03,55.95
4,4,600.0,22.32,46.07,58.61,0.0,0.0,70.78,135108,13.1,...,0.0,2.6,8.8,6,5,83,3794,58.92,57.11,54.95


In [5]:
# data=pd.DataFrame(df[['distance','pp_start','speed1','speed2','speed3','speedf','pp_finish']])
data=pd.DataFrame(df[['distance','age','jockey','trainer','speed_rating','odds','pp_start','speed1','speed2','speedf','pp_finish']])
data.head()

,distance,age,jockey,trainer,speed_rating,odds,pp_start,speed1,speed2,speedf,pp_finish
0,600.0,3,161943,267241,69,96.3,6,57.46,56.42,54.25,7
1,600.0,3,74725,518,86,35.8,7,55.30,55.81,55.10,4
2,600.0,3,122644,83,101,0.5,4,59.10,57.30,55.89,2
3,600.0,3,114312,26392,102,2.9,1,58.53,57.03,55.95,1
4,600.0,4,135108,3794,83,13.1,5,58.92,57.11,54.95,6


In [6]:
data.to_csv('output/20180722.csv')